In [31]:
import duckdb
import os
import pandas as pd
from tabulate import tabulate

# ---------------------------------------------------
# CONFIGURAÇÕES DE DISPLAY (opcional, mas recomendado)
# ---------------------------------------------------
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

# ---------------------------------------------------
# CONFIGURAÇÕES
# ---------------------------------------------------

LOGS_PATH = "/home/ferreirinha/Área de trabalho/tcc_duck_db_vs_spark_one_single/logs/*.csv"

# Conexão DuckDB (em memória)
con = duckdb.connect()

# ---------------------------------------------------
# PARTE 1 — CRIAÇÃO DA VIEW
# ---------------------------------------------------

con.execute(f"""
    CREATE OR REPLACE VIEW logs AS
    SELECT
        *,
    regexp_extract(filename, '[^/]+$') AS source_file,
    replace(
        regexp_extract(filename, '[^/]+$'),
        '.csv',
        ''
    ) AS file
    FROM read_csv_auto(
        '{LOGS_PATH}',
        union_by_name = true
    )
""")

print("✅ View 'logs' criada com sucesso")

# ---------------------------------------------------
# PARTE 2 — CONSULTAS (SQL PURO)
# ---------------------------------------------------

df = con.execute("""
WITH execucoes AS (
    SELECT
        CASE
            WHEN script_name ILIKE 'duckdb%' THEN 'DuckDB'
            WHEN script_name ILIKE 'spark%'  THEN 'Spark'
            WHEN script_name ILIKE 'pandas%' THEN 'Pandas'
        END AS tecnologia,
        CASE
            WHEN script_name ILIKE '%incremental%' THEN 'incremental'
            WHEN script_name ILIKE '%full%'  THEN 'full_load'
        END AS estrategia,
    CAST(regexp_extract(file, '(\\d+)GB$', 1) AS INTEGER) AS tamanho_dado,
    COUNT(*) AS qtd_de_segundos,
    MIN(cpu_usage_percent) AS min_cpu_usage_percent,
    MAX(cpu_usage_percent) AS max_cpu_usage_percent ,
    AVG(cpu_usage_percent) AS avg_cpu_usage_percent,
                 
    MIN(container_memory_percent) AS min_container_memory_percent,
    MAX(container_memory_percent) AS max_container_memory_percent,
    AVG(container_memory_percent) AS avg_container_memory_percent
                                 
    FROM logs
    WHERE DATE(timestamp) >= '2025-12-25'
    GROUP BY ALL
                 
    ORDER BY 3 ASC, 2 ASC, 1 ASC
),

tecnologias AS (

    -- 🔹 Classificação da tecnologia
    SELECT
    tecnologia,
    avg_cpu_usage_percent,
    avg_container_memory_percent

    FROM execucoes
)

-- 🔹 MÉDIA DAS MÉDIAS
SELECT
    tecnologia,
    ROUND(AVG(avg_cpu_usage_percent), 2)     AS cpu_media_percent,
    ROUND(AVG(avg_container_memory_percent), 2) AS memoria_media_percent
FROM tecnologias
GROUP BY tecnologia
ORDER BY tecnologia;
""").fetchdf()

print("\n📊 Amostra dos registros (10 linhas):\n")

print(
    tabulate(
        df,
        headers="keys",
        tablefmt="github",
        showindex=False,
        floatfmt=".2f"
    )
)

# ---------------------------------------------------
# FINALIZAÇÃO
# ---------------------------------------------------

con.close()
print("\n🏁 Execução finalizada")


✅ View 'logs' criada com sucesso

📊 Amostra dos registros (10 linhas):

| tecnologia   |   cpu_media_percent |   memoria_media_percent |
|--------------|---------------------|-------------------------|
| DuckDB       |               17.86 |                   76.06 |
| Pandas       |               18.53 |                   72.87 |
| Spark        |               43.51 |                   94.10 |

🏁 Execução finalizada
